In [1]:
import joblib
from src import Utils, EnsembleFunctions, BERT
import pandas as pd
import numpy as np

/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available = lambda: False

In [3]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [4]:
BertModel = BERT.BertModel

def load_model(model_name: str):
  try:
    return joblib.load(model_name)
  except FileNotFoundError:
    print("ERROR: Model not found")
    return None

learner_bayes = load_model('model_bayes/Bayes.pkl')
learner_lstm = load_model('model_lstm/LSTM.pkl')
learner_bert = load_model('model_bert/mBERT.pkl')
learner_lr = load_model('model_lr/LR.pkl')

/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_lg' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn

In [5]:
learner_bayes

Pipeline(steps=[('tfidf', CountVectorizer()),
                ('bayes', BernoulliNB(alpha=0.3))])

In [6]:
learner_lstm

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 250, batch_first=True)
    (fc): Linear(in_features=250, out_features=2, bias=True)
  ),
))])

In [7]:
learner_bert

Pipeline(steps=[('tokenizer',
                 HuggingfacePretrainedTokenizer(max_length=512,
                                                tokenizer='bert-base-multilingual-cased')),
                ('bert',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=BertModel(
    (bert): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_e...
                (dense): Linear(in_features=3072, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
        )
        (pooler): BertPooler(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (activation): Tanh()
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (classifier): Linear(in_features=768, out_features=2, bias=True)
    )
  ),
))])

In [8]:
learner_lr

LogisticRegression(C=0.0001)

In [9]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')

dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [10]:
TEST_SIZE = 0.1

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [11]:
VAL_SPLIT = 0.5

X_val, X_test, y_val, y_test = Utils.get_train_test_split(
  pd.DataFrame({
    'text': X_test,
    'label': y_test,
  }), 
  VAL_SPLIT,
)

In [12]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

In [13]:
hard_voting_results = EnsembleFunctions.hard_voting(
  learner_bayes, 
  learner_lstm, 
  learner_bert, 
  X_test,
)

hard_voting_results

after count vocab
34293
  (0, 7452)	1
  (0, 7527)	1
  (0, 11900)	1
  (0, 18333)	1
  (0, 18407)	1
  (0, 22118)	1
  (0, 32246)	1
  (0, 37801)	2
  (1, 21044)	1
  (1, 25658)	1
  (1, 31832)	1
  (1, 34063)	1
  (2, 1433)	1
  (2, 1712)	1
  (2, 1773)	1
  (2, 2423)	1
  (2, 2665)	1
  (2, 4885)	2
  (2, 15002)	1
  (2, 16774)	1
  (2, 17642)	1
  (2, 17647)	1
  (2, 18185)	2
  (2, 23045)	1
  (2, 24133)	1
  :	:
  (1419, 1625)	1
  (1419, 3873)	1
  (1419, 5120)	1
  (1419, 5205)	1
  (1419, 5306)	1
  (1419, 6457)	1
  (1419, 10783)	2
  (1419, 11571)	1
  (1419, 25658)	1
  (1419, 28149)	1
  (1419, 30687)	1
  (1419, 31264)	1
  (1419, 31267)	1
  (1419, 33146)	1
  (1419, 38716)	1
  (1420, 834)	1
  (1420, 1244)	1
  (1420, 3873)	1
  (1420, 22868)	1
  (1420, 25262)	1
  (1420, 32107)	1
  (1420, 35276)	1
  (1421, 7296)	1
  (1421, 18185)	1
  (1421, 33299)	1
10957


KeyboardInterrupt: 

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(hard_voting_results, y_test),
  'recall': recall_score(hard_voting_results, y_test),
  'precision': precision_score(hard_voting_results, y_test),
  'f1': f1_score(hard_voting_results, y_test),
}])

,accuracy,recall,precision,f1
0,0.846046,0.818822,0.891911,0.853805


In [ ]:
soft_voting_results = EnsembleFunctions.soft_voting(
  learner_bayes, 
  learner_lstm, 
  learner_bert, 
  X_test,
)

soft_voting_results

array([[0.15941007, 0.84058996],
       [0.24451572, 0.75548428],
       [0.99233688, 0.00766312],
       ...,
       [0.16353516, 0.83646486],
       [0.1014011 , 0.89859887],
       [0.65254836, 0.34745166]])

In [ ]:
soft_voting_results_discrete = np.argmax(soft_voting_results, axis=1)

soft_voting_results_discrete

array([1, 1, 0, ..., 1, 1, 0])

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(soft_voting_results_discrete, y_test),
  'recall': recall_score(soft_voting_results_discrete, y_test),
  'precision': precision_score(soft_voting_results_discrete, y_test),
  'f1': f1_score(soft_voting_results_discrete, y_test),
}])

,accuracy,recall,precision,f1
0,0.846397,0.821821,0.887727,0.853503


In [ ]:
stacking_results = EnsembleFunctions.stacking(
  learner_bayes,
  learner_lstm,
  learner_bert,
  learner_lr,
  X_test
)

stacking_results

array([[0.0935605 , 0.9064395 ],
       [0.07668313, 0.92331687],
       [0.98238279, 0.01761721],
       ...,
       [0.05485927, 0.94514073],
       [0.05103099, 0.94896901],
       [0.96259484, 0.03740516]])

In [ ]:
stacking_results_discrete = np.argmax(stacking_results, axis=1)

stacking_results_discrete

array([1, 1, 0, ..., 1, 1, 0])

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(stacking_results_discrete, y_test),
  'recall': recall_score(stacking_results_discrete, y_test),
  'precision': precision_score(stacking_results_discrete, y_test),
  'f1': f1_score(stacking_results_discrete, y_test),
}])

,accuracy,recall,precision,f1
0,0.849912,0.846524,0.857741,0.852096
